In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/23 13:18:49 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/23 13:18:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 13:19:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sparkCustomer = spark.read.option('header', True).option('delimiter', ',').csv('Dataset.csv')
pandasCustomer = pd.read_csv('Dataset.csv')

sparkCustomer.show()
pandasCustomer.head()

+---+----+----------+
|id |name|referee_id|
+---+----+----------+
|  1|Will|      NULL|
|  2|Jane|      NULL|
|  3|Alex|         2|
|  4|Bill|      NULL|
|  5|Zack|         1|
|  6|Mark|         2|
+---+----+----------+



,id,name,referee_id
0,1,Will,NaN
1,2,Jane,NaN
2,3,Alex,2.0
3,4,Bill,NaN
4,5,Zack,1.0


In [3]:
def find_customer_referee(customer: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    customer.createOrReplaceTempView('Customer')
    sqlQuery = \
    '''
    SELECT name FROM Customer 
    WHERE referee_id != 2 OR referee_id IS NULL
    '''
    customer_name = spark.sql(sqlQuery = sqlQuery)
    return customer_name

customer_name = find_customer_referee(sparkCustomer)
customer_name.show()

+----+
|name|
+----+
|Will|
|Jane|
|Bill|
|Zack|
+----+



In [4]:
def find_customer_referee(customer: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    customer_name = customer.filter((customer['referee_id'] != 2) | 
                                    (customer['referee_id']).isNull())\
                            .select('name')
    return customer_name

customer_name = find_customer_referee(sparkCustomer)
customer_name.show()

+----+
|name|
+----+
|Will|
|Jane|
|Bill|
|Zack|
+----+



In [5]:
def find_customer_referee(customer: pd.DataFrame) -> pd.DataFrame:
    customer_name = customer[(customer['referee_id'] != 2) | 
                             (customer['referee_id'].isna())]\
                            .name
    customer_name = pd.DataFrame(customer_name)
    return customer_name

customer_name = find_customer_referee(pandasCustomer)
customer_name.head()

,name
0,Will
1,Jane
3,Bill
4,Zack
